In [1]:
import os
import re
import math
import ROOT
import traceback
import time
import numpy as np
from ScopeProcessing import Waveform
from array import array

Welcome to JupyROOT 6.22/08


In [ ]:
date = "20211212"
runs = range(494)
for run in runs:
    run_number = "0"*(4-len(str(run)))+str(run)
    run_number = "run_"+run_number
    path = "../Data/"+date+"/"+run_number+"/"
    if not os.path.exists(path):
        print(f"No path: '{path}', next run")
        continue
    list_of_files = [name for name in os.listdir(path) if re.search(r'C3.*\.trc', name)]
    list_of_files.sort()
    print(f"Date: {date}, run #: {run_number}")

    event_number = 0
    
    root_file = ROOT.TFile("../"+date+"_"+run_number+".root","recreate")
    tree = ROOT.TTree("tree",date+"_"+run_number)
    
    Baseline = array("f", [0.])
    Integral = array("f", [0.])
    Start_time = array("i", [0])
    Amplitude = array("f", [0.])
    Integral_23 = array("f", [0.])
    Integral_80 = array("f", [0.])
    Rising_time = array("i", [0])
    Event = array("i", [0])
    Run = array("i", [0])
    Time = array("f", [0.])
    Pulses_number = array("i", [0])
    max_pulses_number = 100
    Pulse_area = array("f", [0. for i in range(max_pulses_number)])
    Pulse_time = array("f", [0. for i in range(max_pulses_number)])
    Pulse_duration = array("f", [0. for i in range(max_pulses_number)])
    
    Baseline_low = array("f", [0.])
    Integral_low = array("f", [0.])
    Amplitude_low = array("f", [0.])
    Integral_80_low = array("f", [0.])
    with_low = array("i", [0])
    
    tree.Branch("run",Run,"run/I")
    tree.Branch("event",Event,"event/I")
    tree.Branch("time",Time,"time/F")
    tree.Branch("rising_time",Rising_time,"rising_time/I")
    tree.Branch("baseline",Baseline,"baseline/F")
    tree.Branch("baseline_low",Baseline_low,"baseline_low/F")
    tree.Branch("integral",Integral,"integral/F")
    tree.Branch("integral_low",Integral_low,"integral_low/F")
    tree.Branch("start_time",Start_time,"start_time/I")
    tree.Branch("amplitude",Amplitude,"amplitude/F")
    tree.Branch("amplitude_low",Amplitude_low,"amplitude_low/F")
    tree.Branch("integral_23",Integral_23,"integral_23/F")
    tree.Branch("integral_80",Integral_80,"integral_80/F")
    tree.Branch("integral_80_low",Integral_80_low,"integral_80_low/F")
    tree.Branch("with_low",with_low,"with_low/I")
    tree.Branch("pulses_number",Pulses_number,"pulses_number/I")
    tree.Branch("pulse_time",Pulse_time,"pulse_time[pulses_number]/F")
    tree.Branch("pulse_area",Pulse_area,"pulse_area[pulses_number]/F")
    tree.Branch("pulse_duration",Pulse_duration,"pulse_duration[pulses_number]/F")
    
    for file in list_of_files:
        Time[0] = os.path.getmtime(path+file)
        file_low = re.sub(r"C3",r"C4",file)
        event_number_from_file = re.search(r"(\d+)\.trc", file)
        Baseline[0] = 0
        Baseline_low[0] = 0
        Integral[0] = 0
        Integral_low[0] = 0
        Start_time[0] = 0
        Amplitude[0] = 0
        Amplitude_low[0] = 0
        Integral_23[0] = 0
        Integral_80[0] = 0
        Integral_80_low[0] = 0
        if event_number%10000 == 0:
            print(f"\t{event_number}")
        if event_number_from_file:
            Event[0] = int(event_number_from_file.group(1))
        else:
            Event[0] = -1
        Run[0] = int(run)
        hist = []
        hist_low = []
        with_low[0] = 0
        try:
            wf = Waveform(path+file)
            hist = wf.amplVec
            hist = np.array(hist)
        except:
            print(f"Problem in {file}")
            print(traceback.format_exc())
            continue
        hist = hist * 1e-3
        
        if os.path.exists(path+file_low):
            with_low[0] = 1
            try:
                wf_low = Waveform(path+file_low)
                hist_low = wf_low.amplVec
                hist_low = np.array(hist_low)
            except:
                print(f"Problem in {file}")
                print(traceback.format_exc())
                continue
            hist_low = hist_low * 1e-3
        else:
            hist_low = hist

        int_hist = np.array(hist, copy=True)
        max_of_integrated_signal = 0
        position_of_int_max = 0
        current_integral = 0
        
        mean_value = hist[:50].mean()
        threshold = 5*np.std(hist[:50]) # порог поиска начала сигнала = 5 стандартных отклонения шума от нулевой линии
        pretrace = []
        pretrace_low = []
        mean_value_low = hist_low[:50].mean()
        integrating_threshold = threshold
        
        for i in range(len(hist)):
            value = hist[i]
            value_low = hist_low[i]
            pretrace.append(value)
            pretrace_low.append(value_low)
            if mean_value-value > threshold:
                pretrace = np.array(pretrace)
                pretrace_low = np.array(pretrace_low)
                mean_value = pretrace.mean()
                mean_value_low = pretrace_low.mean()
                integrating_threshold = np.std(pretrace)
                if i > 0:
                    while mean_value - value > integrating_threshold:
                        pretrace = np.delete(pretrace, i)
                        pretrace_low = np.delete(pretrace_low, i)
                        i -= 1
                        value = hist[i]
                        mean_value = pretrace.mean()
                        integrating_threshold = np.std(pretrace)
                    Start_time[0] = i + 1
                    Baseline[0] = pretrace.mean()
                    Baseline_low[0] = pretrace_low.mean()
                break
        
        for i in range(len(hist)):
            current_integral += hist[i] - Baseline[0]
            int_hist[i] = current_integral
            if current_integral < max_of_integrated_signal:
                max_of_integrated_signal = current_integral
                position_of_int_max = i
        
        level_of_10percent = max_of_integrated_signal/10.
        level_of_90percent = max_of_integrated_signal*0.9
        
        begin_of_signal_rising = 0
        end_of_signal_rising = 0
        Rising_time[0] = 0
        
        for i in range(len(int_hist)):
            if int_hist[i] <= level_of_10percent and not begin_of_signal_rising:
                begin_of_signal_rising = i
            if int_hist[i] <= level_of_90percent and not end_of_signal_rising:
                end_of_signal_rising = i
                Rising_time[0] = end_of_signal_rising - begin_of_signal_rising
                break
        
        if len(pretrace) > 0:
            integrating_threshold = np.std(pretrace)
            threshold = 3*np.std(pretrace) # порог поиска импульсов для интегрирования = 3 стандартных отклонения шума от нулевой линии до начала сигнала
        Amplitude[0] = Baseline[0]
        Amplitude_low[0] = Baseline_low[0]
        is_in_pulse = 1
        Pulses_number[0] = 0
        for j in range (max_pulses_number):
            Pulse_area[j] = 0
            Pulse_time[j] = 0
            Pulse_duration[j] = 0
        i = Start_time[0]
        while i < len(hist):
            value = hist[i]
            value_low = hist_low[i]
            if is_in_pulse:
                if value < Amplitude[0]:
                    Amplitude[0] = value
                if value_low < Amplitude_low[0]:
                    Amplitude_low[0] = value_low
                if Baseline[0] - value < integrating_threshold:
                    is_in_pulse = 0
                    Pulse_duration[Pulses_number[0]] = i - Pulse_time[Pulses_number[0]]
                    Pulses_number[0] += 1
                else:
                    delta = Baseline[0] - value
                    delta_low = Baseline_low[0] - value_low
                    Integral[0] += delta
                    Integral_low[0] += delta_low
                    time_gap = i - Start_time[0]
                    if Pulses_number[0] < max_pulses_number:
                        Pulse_area[Pulses_number[0]] += delta
                    if time_gap < 80:
                        Integral_80[0] += delta
                        Integral_80_low[0] += delta_low
                        if time_gap < 23:
                            Integral_23[0] += delta
            else:
                if Baseline[0] - value > threshold:
                    is_in_pulse = 1
                    while Baseline[0] - value > integrating_threshold:
                        i -= 1
                        value = hist[i]
                    if Pulses_number[0] < max_pulses_number:
                        Pulse_time[Pulses_number[0]] = i+1
            i += 1
        Amplitude[0] = Baseline[0]-Amplitude[0]
        Amplitude_low[0] = Baseline_low[0]-Amplitude_low[0]
        event_number += 1
        tree.Fill()
    root_file.Write()
    root_file.Close()
print("Successfuly completed")

Date: 20211212, run #: run_0000
	0
Date: 20211212, run #: run_0001
	0
Date: 20211212, run #: run_0002
	0
Date: 20211212, run #: run_0003
	0
Date: 20211212, run #: run_0004
	0
Date: 20211212, run #: run_0005
	0
Date: 20211212, run #: run_0006
	0
Date: 20211212, run #: run_0007
	0
Date: 20211212, run #: run_0008
	0
Date: 20211212, run #: run_0009
	0
Date: 20211212, run #: run_0010
	0
Date: 20211212, run #: run_0011
	0
Date: 20211212, run #: run_0012
	0
Date: 20211212, run #: run_0013
	0
Date: 20211212, run #: run_0014
	0
Date: 20211212, run #: run_0015
	0
